<a href="https://colab.research.google.com/github/Data-Science-and-Data-Analytics-Courses/MITx---Machine-Learning-with-Python-From-Linear-Models-to-Deep-Learning-Jun-11-2019/blob/master/Unit%201%20Linear%20Classifiers%20and%20Generalizations%20(2%20weeks)/Project%201%3A%20Automatic%20Review%20Analyzer/Project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 1: Automatic Review Analyzer

---



## Packages and libraries

In [0]:
import sys

url = "https://github.com/Data-Science-and-Data-Analytics-Courses/Notebook-Library"

!git clone "{url}" "/nblib"
%run "/nblib/.Importable.ipynb"
sys.path.append("/")

Cloning into '/nblib'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 26 (delta 12), reused 7 (delta 2), pack-reused 0
Unpacking objects: 100% (26/26), done.


In [0]:
from google.colab import drive
from nblib import Git

URL = "https://github.com/Data-Science-and-Data-Analytics-Courses/MITx---Machine-Learning-with-Python-From-Linear-Models-to-Deep-Learning-Jun-11-2019"
nbdir_rel = "Unit 1 Linear Classifiers and Generalizations (2 weeks)/Project 1: Automatic Review Analyzer/"

REPO = Git.clone(URL)
NBDIR = REPO/nbdir_rel
os.chdir(NBDIR)
sys.path.append(NBDIR.as_posix())

importing Jupyter notebook from /nblib/Git.ipynb
Cloning into '/content/MITx---Machine-Learning-with-Python-From-Linear-Models-to-Deep-Learning-Jun-11-2019'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 165 (delta 6), reused 0 (delta 0), pack-reused 149
Receiving objects: 100% (165/165), 1.24 MiB | 16.87 MiB/s, done.
Resolving deltas: 100% (76/76), done.


In [0]:
from sentiment_analysis.project1 import get_order

from pathlib import Path
import numpy as np

## Hinge Loss


In [0]:
def hinge_loss_single(feature_vector, label, theta, theta_0):
    """
    Finds the hinge loss on a single data point given specific classification
    parameters.

    Args:
        feature_vector - A numpy array describing the given data point.
        label - A real valued number, the correct classification of the data
            point.
        theta - A numpy array describing the linear classifier.
        theta_0 - A real valued number representing the offset parameter.


    Returns: A real number representing the hinge loss associated with the
    given data point and parameters.
    """
    # Your code here
    pred = np.dot(theta, feature_vector) + theta_0
    loss = max(0, 1-label*pred)
    
    return loss
  
feature_vector = np.array([1, 2])
label, theta, theta_0 = 1, np.array([-1, 1]), -0.2
exp_res = 1 - 0.8
assert exp_res == hinge_loss_single(feature_vector, label, theta, theta_0)

In [0]:
def hinge_loss_full(feature_matrix, labels, theta, theta_0):
    """
    Finds the total hinge loss on a set of data given specific classification
    parameters.

    Args:
        feature_matrix - A numpy matrix describing the given data. Each row
            represents a single data point.
        labels - A numpy array where the kth element of the array is the
            correct classification of the kth row of the feature matrix.
        theta - A numpy array describing the linear classifier.
        theta_0 - A real valued number representing the offset parameter.


    Returns: A real number representing the hinge loss associated with the
    given dataset and parameters. This number should be the average hinge
    loss across all of the points in the feature matrix.
    """
    # Your code here
    predictions = np.dot(feature_matrix, theta) + theta_0
    losses = np.maximum(np.zeros(len(predictions)), 1 - labels * predictions)
    loss = np.mean(losses)
    
    return loss

feature_matrix = np.array([[1, 2], [1, 2]])
labels, theta, theta_0 = np.array([1, 1]), np.array([-1, 1]), -0.2
exp_res = 1 - 0.8
assert exp_res == hinge_loss_full(feature_matrix, labels, theta, theta_0)

## Perceptron Algorithm


In [0]:
def perceptron_single_step_update(
        feature_vector,
        label,
        current_theta,
        current_theta_0):
    """
    Properly updates the classification parameter, theta and theta_0, on a
    single step of the perceptron algorithm.

    Args:
        feature_vector - A numpy array describing a single data point.
        label - The correct classification of the feature vector.
        current_theta - The current theta being used by the perceptron
            algorithm before this update.
        current_theta_0 - The current theta_0 being used by the perceptron
            algorithm before this update.

    Returns: A tuple where the first element is a numpy array with the value of
    theta after the current update has completed and the second element is a
    real valued number with the value of theta_0 after the current updated has
    completed.
    """
    # Your code here
    theta = np.array(current_theta)
    theta_0 = current_theta_0
    if label * (theta.dot(feature_vector) + theta_0) <= 0: # misclassified
      theta += label * feature_vector # update theta
      theta_0 += label # update theta 0
    
    return theta, theta_0
    
feature_vector = np.array([1, 2])
label, theta, theta_0 = 1, np.array([-1, 1]), -1.5
exp_theta, exp_theta_0 = (np.array([0, 3]), -0.5)
upd_theta, upd_theta_0 = perceptron_single_step_update(feature_vector, label, theta, theta_0)
assert np.all(upd_theta == exp_theta)
assert upd_theta_0 == exp_theta_0

feature_vector = np.array([1, 2])
label, theta, theta_0 = 1, np.array([-1, 1]), -1
exp_theta, exp_theta_0 = (np.array([0, 3]), 0)
upd_theta, upd_theta_0 = perceptron_single_step_update(feature_vector, label, theta, theta_0)
assert np.all(upd_theta == exp_theta)
assert upd_theta_0 == exp_theta_0

In [0]:
def perceptron(feature_matrix, labels, T):
    """
    Runs the full perceptron algorithm on a given set of data. Runs T
    iterations through the data set, there is no need to worry about
    stopping early.

    NOTE: Please use the previously implemented functions when applicable.
    Do not copy paste code from previous parts.

    NOTE: Iterate the data matrix by the orders returned by get_order(feature_matrix.shape[0])

    Args:
        feature_matrix -  A numpy matrix describing the given data. Each row
            represents a single data point.
        labels - A numpy array where the kth element of the array is the
            correct classification of the kth row of the feature matrix.
        T - An integer indicating how many times the perceptron algorithm
            should iterate through the feature matrix.

    Returns: A tuple where the first element is a numpy array with the value of
    theta, the linear classification parameter, after T iterations through the
    feature matrix and the second element is a real number with the value of
    theta_0, the offset classification parameter, after T iterations through
    the feature matrix.
    """
    # Your code here
    theta, theta_0 = np.zeros(feature_matrix.shape[1]), 0
    for t in range(T):
        for i in get_order(feature_matrix.shape[0]):
            # Your code here
            if labels[i] * (theta.dot(feature_matrix[i]) + theta_0) <= 0: # misclassified
              theta += labels[i] * feature_matrix[i] # update theta
              theta_0 += labels[i] # update theta 0
              
    return theta, theta_0

def test_perceptron(feature_matrix, labels, T, exp_theta, exp_theta_0):
  theta, theta_0 = perceptron(feature_matrix, labels, T)
  assert np.all(theta == exp_theta)
  assert theta_0 == exp_theta_0
  
feature_matrix = np.array([[1, 2]])
labels = np.array([1])
T = 1
exp_theta, exp_theta_0 = (np.array([1, 2]), 1)
test_perceptron(feature_matrix, labels, T, exp_theta, exp_theta_0)

feature_matrix = np.array([[1, 2], [-1, 0]])
labels = np.array([1, 1])
T = 1
exp_theta, exp_theta_0 = (np.array([0, 2]), 2)
test_perceptron(feature_matrix, labels, T, exp_theta, exp_theta_0)

feature_matrix = np.array([[1, 2]])
labels = np.array([1])
T = 2
exp_theta, exp_theta_0 = (np.array([1, 2]), 1)
test_perceptron(feature_matrix, labels, T, exp_theta, exp_theta_0)

feature_matrix = np.array([[1, 2], [-1, 0]])
labels = np.array([1, 1])
T = 2
exp_theta, exp_theta_0 = (np.array([0, 2]), 2)
test_perceptron(feature_matrix, labels, T, exp_theta, exp_theta_0)

In [0]:
def average_perceptron(feature_matrix, labels, T):
    """
    Runs the average perceptron algorithm on a given set of data. Runs T
    iterations through the data set, there is no need to worry about
    stopping early.

    NOTE: Please use the previously implemented functions when applicable.
    Do not copy paste code from previous parts.

    NOTE: Iterate the data matrix by the orders returned by get_order(feature_matrix.shape[0])


    Args:
        feature_matrix -  A numpy matrix describing the given data. Each row
            represents a single data point.
        labels - A numpy array where the kth element of the array is the
            correct classification of the kth row of the feature matrix.
        T - An integer indicating how many times the perceptron algorithm
            should iterate through the feature matrix.

    Returns: A tuple where the first element is a numpy array with the value of
    the average theta, the linear classification parameter, found after T
    iterations through the feature matrix and the second element is a real
    number with the value of the average theta_0, the offset classification
    parameter, found after T iterations through the feature matrix.

    Hint: It is difficult to keep a running average; however, it is simple to
    find a sum and divide.
    """
    # Your code here
    theta, theta_0 = np.zeros(feature_matrix.shape[1]), 0
    theta_sum, theta_0_sum = np.zeros(feature_matrix.shape[1]), 0
    for t in range(T):
        for i in get_order(feature_matrix.shape[0]):
            # Your code here
            if labels[i] * (theta.dot(feature_matrix[i]) + theta_0) <= 0: # misclassified
              theta += labels[i] * feature_matrix[i] # update theta
              theta_0 += labels[i] # update theta 0
            theta_sum += theta
            theta_0_sum += theta_0
    
    theta_avg = theta_sum / (feature_matrix.shape[0] * T)
    theta_0_avg = theta_0_sum / (feature_matrix.shape[0] * T)
              
    return theta_avg, theta_0_avg

def test_average_perceptron(feature_matrix, labels, T, exp_theta, exp_theta_0):
  theta_avg, theta_0_avg = average_perceptron(feature_matrix, labels, T)
  assert np.all(theta_avg == exp_theta)
  assert theta_0_avg == exp_theta_0

feature_matrix = np.array([[1, 2]])
labels = np.array([1])
T = 1
exp_theta, exp_theta_0 = (np.array([1, 2]), 1)
test_average_perceptron(feature_matrix, labels, T, exp_theta, exp_theta_0)

feature_matrix = np.array([[1, 2], [-1, 0]])
labels = np.array([1, 1])
T = 1
exp_theta, exp_theta_0 = (np.array([-0.5, 1]), 1.5)
test_average_perceptron(feature_matrix, labels, T, exp_theta, exp_theta_0)

feature_matrix = np.array([[1, 2]])
labels = np.array([1])
T = 2
exp_theta, exp_theta_0 = (np.array([1, 2]), 1)
test_average_perceptron(feature_matrix, labels, T, exp_theta, exp_theta_0)

feature_matrix = np.array([[1, 2], [-1, 0]])
labels = np.array([1, 1])
T = 2
exp_theta, exp_theta_0 = (np.array([-0.25, 1.5]), 1.75)
test_average_perceptron(feature_matrix, labels, T, exp_theta, exp_theta_0)

## Pegasos Algorithm
[Pegasos: Primal Estimated sub-GrAdient
SOlver for SVM](https://prod-edxapp.edx-cdn.org/assets/courseware/v1/16f13f7ac37ae86ebe0372f2410bcec4/asset-v1:MITx+6.86x+1T2019+type@asset+block/resources_pegasos.pdf)  


In [0]:
def pegasos_single_step_update(
        feature_vector,
        label,
        L,
        eta,
        current_theta,
        current_theta_0):
    """
    Properly updates the classification parameter, theta and theta_0, on a
    single step of the Pegasos algorithm

    Args:
        feature_vector - A numpy array describing a single data point.
        label - The correct classification of the feature vector.
        L - The lamba value being used to update the parameters.
        eta - Learning rate to update parameters.
        current_theta - The current theta being used by the Pegasos
            algorithm before this update.
        current_theta_0 - The current theta_0 being used by the
            Pegasos algorithm before this update.

    Returns: A tuple where the first element is a numpy array with the value of
    theta after the current update has completed and the second element is a
    real valued number with the value of theta_0 after the current updated has
    completed.
    """
    # Your code here
    raise NotImplementedError
